# MP305 Network Flow Models I - Michael Tuite

## Overview 

This file contains a number of Python functions for finding the maximal flow through a network $G$ subject to minimal cost using the Ford Fulkerson Algorithm.

The network graph $G$ is stored in a set `G` of two element tuples `(i,j)` describing the directed arcs $(i,j)$ of $G$.

It is assumed that node number $1$ is the source and the greatest node `Nsink` is the sink.  
Thus `G={(1,2),(2,3),(1,3)}`  describes a network with 3 nodes where node 1 is the source and node 3 is the sink.

The capacity, flow and cost of the arc `(i,j)` of `G` are stored in `c[i][j]`, `phi[i][j]` and `l[i][j]`. 
Here `c`, `phi` and `l` are Python lists.

## Python Functions
### The `Initialise(G)` function
Having defined the network `G`, initialise `c`, `phi` and `l` values to zero via the `Initialise` function before defining their values in any particular example.  The global variable `Nsink`,  the sink node of `G`, is also found by the `Initialise` function .

### The main `Iterate(G)`function 
This implements the full algorithm to find the maximum flow with minimal cost. 

## The `Iterate(G)` function is based on a number of other Python functions:

### `Flows(G)` 
This checks for conservation of flow and prints out all of the current flows for G and the total cost of this flow. 

###  `Links(G)`
This finds all arcs `(i,j1)`, ` (i,j2)`,  ... *out* of node `i` of `G`.  The nodes `j1,j2,..` are stored in a global list of sets `Out`.

###  `SourceSink(G)`
This finds all of the paths from source to sink in any network `G` and results in a global set `SinkPaths` of such paths.

###  `IncremNet(G)`
This finds the Incremental Network `Gp` associated with the current flow of the network `G`.

###  `Newflows(G)`
This updates the flows `phi` of `G` according to the best chain found through `Gp`. If the maximal flow is reached, then this is indicated and the maximum flow value is outputed. Otherwise, the output is: the change in flow (`eps`), the cost of the best chain, and the best chain.

###  `Iterate(G)`
Implements the full algorithm to find the maximum flow with minimal cost. 
The output is as follows:

(1) The incremental network `Gp`.

(2) The paths through `Gp` from source to sink. 

(3) The output of `Newflows(G)`. 

(4) The output of `Flows(G)` giving the current flows and cost of `G`.


In [ ]:
def Initialise (Gin):
    global c,phi,l,cp,lp,Nsink
    Nsink=1
    for arc in Gin:
        i,j=arc
        Nsink=max(Nsink,i,j)
    # for convenience c[i][j] is capacity of arc [i,j]
    c=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    phi=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    l=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    print("All values of c,phi and l initialised to zero")


In [ ]:
def Flows (Gin):
    global Nsink,l,phi
    Flowin=[0 for i in range(Nsink+1)]
    Flowout=[0 for i in range(Nsink+1)]
    for arc in Gin:
        i,j=arc
        Flowin[j] = Flowin[j] + phi[i][j]
        Flowout[i] = Flowout[i] + phi[i][j]
    for k in range(2,Nsink):
        if Flowin[k] != Flowout[k]:
            print("*** ERROR *** Flow not conserved at node", k)
    if Flowout[1] != Flowin[Nsink]:
        print("*** ERROR *** Flow not conserved at source or sink")
    Totalcost = 0
    for arc in G:
        i,j=arc
        phi_ij = phi[i][j]
        Totalcost = Totalcost + l[i][j]*phi_ij
        print(arc," has flow ",phi_ij)
    print("Total Cost is ", Totalcost)


In [ ]:
def Links (Gin):
    global Nsink,Out
    Out=[set() for k in range(Nsink)] # labelled 0..Nsink-1
    for arc in Gin:
        i,j = arc
        Out[i - 1] = Out[i - 1] | set([j])

In [ ]:
def SourceSink(Gin):
# finds all paths SinkPaths from source 1 to sink Nsink of network G 
    global Nsink,SinkPaths
    Links(Gin)
    Paths = set() # current paths from source stored as set of tuples
    SinkPaths = set() # paths from source to sink Nsink stored as set of tuples
    path = 1 # source node label
    for node in Out[0]:# need out edge from node 1
        pathn = (path,node)
        if node == Nsink:
            SinkPaths = SinkPaths | set([pathn])
        else:
            Paths = Paths | set([pathn])
    Npaths = len(Paths)
    while (0 < Npaths):
        NewPaths = set()
        for oldpath in Paths:
            nold = len(oldpath)
            m = oldpath[-1] # last node in tuple oldpath
            for mout in Out[m-1]:
                if not mout in oldpath:
                    if mout == Nsink:
                        SinkPaths = SinkPaths | set([oldpath+tuple([Nsink])])
                    else:
                        NewPaths = NewPaths | set([oldpath+tuple([mout])])
        Paths = NewPaths
        Npaths = len(Paths)
    print("Paths from source to sink: ",SinkPaths)

In [ ]:
def IncremNet(Gin):
# procedure to create incremental network Gp from given flow network G 
    global Gp,Nsink,phi,c,l,cp,lp,ArcSign
# define lists for ArcSign, cp and lp  (indexed by 0..Nsink-1)
    cp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    lp=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    ArcSign=[[0 for i in range(Nsink+1)] for j in range(Nsink+1)]
    Gp=set([])
    for arc in Gin:
        i,j=arc
        pij = phi[i][j]; pji = phi[j][i]; cij = c[i][j]; lij = l[i][j]
        if (pij < cij and (pji == 0 or not (j,i) in Gin)): # ij arc
            #Gp edges, capacitites and costs added
            cpij = cij - pij; cp[i][j] = cpij; lpij = lij; lp[i][j] =lpij
            ArcSign[i][j] = 1
            Gp=Gp | {(i,j)}
        if pij>0: # ji arc
            cpji = pij; cp[j][i] = cpji; lpji=-lij; lp[j][i] = lpji
            ArcSign[j][i] = -1
            Gp=Gp | {(j,i)}
    print("Incremental Network:",Gp)

In [ ]:
def Newflows(Gin):
# A procedure to modify original flows on Gin along SinkPaths of Gp with minimal cost
    global Gp,phi,c,l,cp,lp,ArcSign,Out
    SourceSink(Gp)
    if SinkPaths == set():
        Links(Gin)
        Flow = 0
        for node in Out[0]:
            Flow = Flow + phi[1][node]
        Cost=0
        for arc in Gin:
            i,j=arc
            Cost=Cost+l[i][j]*phi[i][j]
        print("Maximal flow found:", Flow, " with minimal cost ", Cost)
    else:
        for k in range(len(SinkPaths)):
            cost = 0
            epset = set()
            path=list(SinkPaths)[k]
            for n in range(0, len(path)-1):
                i = path[n]; j = path[n+1];  epset = epset | set([cp[i][j]]); cost = lp[i][j] + cost
            eps = min(tuple(epset))
            if k == 0: # first path
                mincost = cost; bestpath = path; besteps = eps
            elif cost < mincost: 
                mincost = cost; bestpath = path; besteps = eps
        print("A best path in Gp is ", bestpath, " of minimum cost ", mincost)
        print("The min capacity on this path is epsilon ", eps)
        print("The min cost is ", mincost)
        for k in range(0, len(bestpath) - 1):
            i = bestpath[k]; j = bestpath[k+1]
            if ArcSign[i][j] == 1:
                phinewij = phi[i][j] + besteps; phi[i][j]=phinewij
            else:
                phinewji=phi[j][i] = phi[j][i] - besteps; phi[j][i]=phinewji
    #print("Flow=",Flow)

In [ ]:
def Iterate(Gin):
    IncremNet(Gin)
    Newflows(Gin)
    for arc in Gin:
        i,j=arc
        print((i,j)," flow = ", phi[i][j])

# Example 1. This is the first example discussed in lectures.

## Define `G` and initialise all capacity, flow, cost values to zero  


In [ ]:
G={(1,2),(1,3),(2,3),(2,4),(2,5),(3,5),(4,6),(5,4),(5,6)}

In [ ]:
Initialise (G)

In [ ]:
Flows(G)

## Define Capacities

In [ ]:
c[1][2] = 4; c[1][3] = 5; c[2][3] = 2  
c[2][4] = 1; c[2][5] = 4; c[3][5] = 3  
c[4][6] = 2; c[5][4] = 2; c[5][6] = 6

## Define Flows

In [ ]:
phi[1][2]=4; phi[1][3]=1; phi[2][3]=2; phi[2][4]=1; phi[2][5]=1
phi[3][5]=3; phi[4][6]=2; phi[5][4]=1; phi[5][6]=3

In [ ]:
Flows(G)

## Use `Iterate` to run one iteration of the Ford Fulkerson Algorithm

In [ ]:
Iterate(G)

In [ ]:
Iterate(G)

## Examples 2&3. Define the network `G` and the capacities `c[i][j]` for the given data and follow the same sequence of steps as for example 1. Note that some arcs are two-way